In [1]:
from src.get_quits_codes import get_bpc_code, get_lpc_code
from src.circuit_from_cx_list import memory_experiment_circuit_from_cx_list
import numpy as np
from src.permute_within_each_stabilizer import random_permutation_within_each_stabilizer
from src.permute_single_stabilizer import permute_single_stabilizer_inplace
from collections import Counter
from src.distance_from_decoder import find_distance
import time
import sinter
from ldpc.sinter_decoders import SinterBpOsdDecoder
from src.cx_list_from_stabilizers_in_sequence import RotatedSurfaceCode
np.random.seed(42)

In [41]:
code = RotatedSurfaceCode(5)
cx_list = code.generate_cx_list()
lz = code.lz
lx = code.lx
ancilla_type, data_mapping, ancilla_mapping, flag_mapping= code.build_mappings()

In [26]:
cx_list, ancilla_type, data_mapping, ancilla_mapping, lz, lx = get_bpc_code(cx_order='theirs')
qubit_to_name = {v:k for k,v in data_mapping.items()}
qubit_to_name.update({v:k for k,v in ancilla_mapping.items()})

In [42]:
# separate the cxs such that they are done one stabilizer at a time
cx_list = sorted(cx_list, key=lambda x: x[1])

In [43]:
p_spam_error = 0.01
p_phenomenological_error = 0.01
p_hook_error = 0.01

In [44]:
stabilizer_weights = Counter(label for _, label in cx_list)
def all_hook_errors_on_ancilla(anc, p=p_hook_error):
    return [(cx_idx,p) for cx_idx in range(1,stabilizer_weights[anc]-2)]
def all_hook_errors(p=p_hook_error):
    return {ancilla: all_hook_errors_on_ancilla(ancilla, p=p) for ancilla, a_type in ancilla_type.items() if a_type == 'X'}

In [45]:
def get_logical_error_rate():
    circ = memory_experiment_circuit_from_cx_list(
                    cx_list=cx_list,
                    ancilla_type=ancilla_type,
                    data_mapping=data_mapping,
                    ancilla_mapping=ancilla_mapping,
                    flag_mapping=dict(),  # No flag mapping used here
                    logicals=lz,
                    logical_type='Z',
                    p_cx=0.0,#05,#
                    p_idle=0.,
                    p_spam_error=p_spam_error,
                    p_phenomenological_error=p_phenomenological_error,
                    hook_errors=all_hook_errors(),
                    number_of_cycles=1,
                    flag=False
                )
    task = sinter.Task(circuit=circ)

    stats = sinter.collect(tasks=[task,],
                           num_workers=10,
                           max_shots=100000,
                           max_errors=1000,
                           decoders=['bposd'],
                           custom_decoders={
                               "bposd": SinterBpOsdDecoder(max_iter=10, osd_order=5, osd_method='OSD_E')}# max_iter=10000,osd_order=15)}
                           )
    for stat in stats:
        logical_error_rate = stat.errors / stat.shots
        logical_error_rate_error = np.sqrt(
            logical_error_rate * (1 - logical_error_rate) / stat.shots)
        break
    else:
        logical_error_rate = 0.0
        logical_error_rate_error = 0.0
    print(logical_error_rate, logical_error_rate_error)

In [46]:
random_permutation_within_each_stabilizer(cx_list)

In [47]:
get_logical_error_rate()

0.00155 0.0001244024718403939


In [48]:
def get_minimal_error():
    circ = memory_experiment_circuit_from_cx_list(
                cx_list=cx_list,
                ancilla_type=ancilla_type,
                data_mapping=data_mapping,
                ancilla_mapping=ancilla_mapping,
                flag_mapping=dict(),  # No flag mapping used here
                logicals=lz,
                logical_type='Z',
                p_cx=0.,
                p_idle=0.,
                p_spam_error=p_spam_error,
                p_phenomenological_error=p_phenomenological_error,
                hook_errors=hook_errors,
                number_of_cycles=1,
                flag=False
            )
    dist, minimal_error, error_tags = find_distance(circ, max_bp_iterations=10000,osd_order=15)
    print(error_tags)
    return [e for e in error_tags if e!='phenomenological'], dist
    # errors = circ.search_for_undetectable_logical_errors(
    #     dont_explore_edges_increasing_symptom_degree=False,
    #     dont_explore_detection_event_sets_with_size_above=6,
    #     dont_explore_edges_with_degree_above=9999,
    #     canonicalize_circuit_errors=True,
    # )
    # qubits_in_minimal_error = [error.circuit_error_locations[0].instruction_targets.targets_in_range[0].gate_target.value for error in errors]
    # hook_ancillas_in_minimal_error = [qubit_to_name[qubit] for qubit in qubits_in_minimal_error if qubit in ancilla_mapping.values()]
    # # print([qubit_to_name[qubit] for qubit in qubits_in_minimal_error])
    # return hook_ancillas_in_minimal_error, len(errors)

In [49]:
# start with hook errors on all ancillas, then keep only hook errors on ancillas in minimal error
for _ in range(1000):
    hook_errors = all_hook_errors()
    bad_ancillas, distance = get_minimal_error()
    # print(bad_ancillas)
    print('full distance: ', distance)
    hook_errors = {ancilla: all_hook_errors_on_ancilla(ancilla) for ancilla in bad_ancillas}
    while True:
        a = np.random.choice(bad_ancillas)
        permute_single_stabilizer_inplace(cx_list, a)
        bad_ancillas, new_distance = get_minimal_error()
        print('new distance: ', new_distance)
        if new_distance > distance:
            break

['X1', 'X0', 'phenomenological']
full distance:  3.0
Changed: (3, 'X1') -> (7, 'X1')
Changed: (2, 'X1') -> (8, 'X1')
Changed: (8, 'X1') -> (2, 'X1')
Changed: (7, 'X1') -> (3, 'X1')
['X1', 'X0', 'phenomenological']
new distance:  3.0
Changed: (8, 'X1') -> (2, 'X1')
Changed: (2, 'X1') -> (8, 'X1')
['phenomenological', 'X0', 'phenomenological', 'phenomenological']
new distance:  4.0
['phenomenological', 'phenomenological', 'X6', 'phenomenological']
full distance:  4.0
Changed: (17, 'X6') -> (22, 'X6')
Changed: (22, 'X6') -> (17, 'X6')
['phenomenological', 'phenomenological', 'phenomenological', 'phenomenological', 'phenomenological']
new distance:  5.0
['phenomenological', 'phenomenological', 'X2', 'phenomenological']
full distance:  4.0
Changed: (7, 'X2') -> (12, 'X2')
Changed: (12, 'X2') -> (7, 'X2')
['phenomenological', 'phenomenological', 'phenomenological', 'phenomenological', 'phenomenological']
new distance:  5.0
['phenomenological', 'X0', 'phenomenological', 'phenomenological']
fu

ValueError: 'a' cannot be empty unless no samples are taken

In [50]:
get_logical_error_rate()

0.00017 4.122755146743498e-05
